# Trips in time and space order
Sorts output trips in time and space order, which is useful for disaggregate (individual) dynamic traffic assignment and person time/space visualization.  Trips in time and space order means the trip origin, destination, and depart period from one trip to the next makes sense.

# Input and output filenames

In [1]:
pipeline_filename = '../test_example_mtc/output/pipeline.h5'
output_trip_filename = "../test_example_mtc/output/final_trips_time_space_order.csv"

# Libraries

In [3]:
import pandas as pd
import numpy as np
import itertools

# Read tables directly from the pipeline

In [4]:
# get tables (if run as mp then trip_mode_choice is final state of the tables) 
pipeline = pd.io.pytables.HDFStore(pipeline_filename)
tours = pipeline['/tours/stop_frequency']
trips = pipeline['/trips/trip_mode_choice']
jtp = pipeline['/joint_tour_participants/joint_tour_participation']

# Add related fields, including joint trip participant ids

In [5]:
trips["tour_participants"] = trips.tour_id.map(tours.number_of_participants)
trips["tour_category"] = trips.tour_id.map(tours.tour_category)
trips["parent_tour_id"] = trips.tour_id.map(tours.index.to_series()).map(tours.parent_tour_id)
trips["tour_start"] = trips.tour_id.map(tours.start)
trips["tour_end"] = trips.tour_id.map(tours.end)
trips["parent_tour_start"] = trips.parent_tour_id.map(tours.start)
trips["parent_tour_end"] = trips.parent_tour_id.map(tours.end)
trips["inbound"] = ~trips.outbound

# Create additional trips records for other persons on joint trips

In [6]:
tour_person_ids = jtp.groupby("tour_id").apply(lambda x: pd.Series({"person_ids": " ".join(x["person_id"].astype("str"))}))
trips = trips.join(tour_person_ids, "tour_id")
trips["person_ids"] = trips["person_ids"].fillna("")
trips.person_ids = trips.person_ids.where(trips.person_ids!="", trips.person_id)
trips["person_ids"] = trips["person_ids"].astype(str)

person_ids = [*map(lambda x: x.split(" "),trips.person_ids.tolist())]
person_ids = list(itertools.chain.from_iterable(person_ids))

trips_expanded = trips.loc[np.repeat(trips.index, trips['tour_participants'])]
trips_expanded.person_id = person_ids

trips_expanded["trip_id"] = trips_expanded.index
trips_expanded["trip_id"] = trips_expanded["trip_id"].astype('complex128') #for larger ids

while trips_expanded["trip_id"].duplicated().any():
  trips_expanded["trip_id"] = trips_expanded["trip_id"].where(~trips_expanded["trip_id"].duplicated(), trips_expanded["trip_id"] + 0.1)

trips_expanded = trips_expanded.sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])

# Pull out at-work trips and put back in at the right spot

In [7]:
atwork_trips = trips_expanded[trips_expanded.tour_category == "atwork"]

trips_expanded_last_trips = trips_expanded[trips_expanded.trip_num == trips_expanded.trip_count]
parent_tour_trips_with_atwork_trips = trips_expanded_last_trips.merge(atwork_trips, left_on="tour_id", right_on="parent_tour_id")
parent_tour_trips_with_atwork_trips["atwork_depart_after"] = parent_tour_trips_with_atwork_trips.eval("depart_y >= depart_x")

parent_trip_id = parent_tour_trips_with_atwork_trips[parent_tour_trips_with_atwork_trips["atwork_depart_after"]]
parent_trip_id.index = parent_trip_id["trip_id_y"]

for person in parent_trip_id["person_id_x"].unique():
    
    person_all_trips = trips_expanded[(trips_expanded["person_id"].astype("str") == person) & (trips_expanded.tour_category != "atwork")]
    
    person_atwork_trips = parent_trip_id[parent_trip_id["person_id_x"].astype("str") == person]
    parent_trip_index = person_all_trips.index.astype('complex128').get_loc(person_atwork_trips.trip_id_x[0]) 
    
    before_trips = person_all_trips.iloc[0:(parent_trip_index+1)]
    after_trips = person_all_trips.iloc[(parent_trip_index+1):]

    person_actual_atwork_trips = atwork_trips[(atwork_trips["person_id"].astype("str") == person)]
    
    new_person_trips = before_trips.append(person_actual_atwork_trips).append(after_trips)
    
    trips_expanded = trips_expanded[~(trips_expanded["person_id"].astype("str") == person)] #remove and add back due to indexing
    trips_expanded = trips_expanded.append(new_person_trips)

# Add fields to verify sorting

In [8]:
trips_expanded["next_person_id"] = trips_expanded["person_id"].shift(-1)
trips_expanded["next_origin"] = trips_expanded["origin"].shift(-1)
trips_expanded["next_depart"] = trips_expanded["depart"].shift(-1)
trips_expanded["spatial_consistent"] = trips_expanded["destination"] == trips_expanded["next_origin"]
trips_expanded["time_consistent"] = trips_expanded["next_depart"] >= trips_expanded["depart"]
trips_expanded["spatial_consistent"].loc[trips_expanded["next_person_id"] != trips_expanded["person_id"]] = True
trips_expanded["time_consistent"].loc[trips_expanded["next_person_id"] != trips_expanded["person_id"]] = True

print("{}\n\n{}".format(trips_expanded["spatial_consistent"].value_counts(), trips_expanded["time_consistent"].value_counts()))


True    489
Name: spatial_consistent, dtype: int64

True    489
Name: time_consistent, dtype: int64


c:\programdata\anaconda3\envs\asimtest\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Write all trips

In [9]:
trips_expanded.to_csv(output_trip_filename)